In [1]:
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from Utils.Dataloader import ML_dataloader
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import time
from datetime import datetime
RANDOM_SEED = 1234

In [2]:
TRAINING_DICT = {"years":[2017, 2018, 2019,], "sites":[1,], "type":"Both"}
TESTING_DICT = {"years":[2020,], "sites":[1,], "type":"Both"}

In [3]:
Parameters = {"n_estimators":[100,500, 1200, 2000], "max_depth":[ 8, 15, 25, 36, None]}


In [4]:
X_train, Y_train = ML_dataloader(sites=TRAINING_DICT["sites"], years=TRAINING_DICT["years"], type=TRAINING_DICT["type"], verbose=True)
Y_train = Y_train.squeeze()
print("X shape: {x_shape}, Y shape: {y_shape}".format(x_shape = X_train.shape, y_shape = Y_train.shape))

assert(np.any(np.isnan(X_train)) == False and np.any(np.isnan(Y_train)) == False)

Loaded 49 images in site 1, year 2017
Loaded 47 images in site 1, year 2018
Loaded 49 images in site 1, year 2019
Completed! Loaded 145 images in total.
X shape: (9502720, 80), Y shape: (9502720, 1)


In [5]:

best_param = None
best_score = 0
for n_estimator in Parameters["n_estimators"]:
  for max_depth in  Parameters["max_depth"]:
      rf = RandomForestClassifier(n_estimators=n_estimator, max_depth=max_depth, random_state=RANDOM_SEED, n_jobs= -2)
      score = cross_val_score(rf, X_train, Y_train, cv=3)
      mean_score = np.mean(score)
      curr_time = datetime.now()
      curr_time_str = curr_time.strftime("%d/%m/%Y %H:%M")
      print("{t}:   n estimator: {n}, max depth: {d}, score: {s}".format(n= n_estimator, d=max_depth, s=mean_score, t=curr_time_str))
      f = open("./Logs/rf_tuning_log.txt","a")
      f.write("{t}:   n estimator: {n}, max depth: {d}, score: {s}, mean_score: {m}\n".format(n= n_estimator, d=max_depth, s=score, t=curr_time_str, m = mean_score))
      f.close()
      if score > best_score:
        print("======Update best score!======")
        best_param = (n_estimator, max_depth)
        best_score = score
print(best_param)
print(best_score)





Fitting 3 folds for each of 20 candidates, totalling 60 fits


C:\Users\11027\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
